# Question 2

#### Data frame with Neighbourhood Latitude and Longitude


In [ ]:
import requests
import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup

In [60]:
url=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup=BeautifulSoup(url,'lxml')

data = []
columns = []
table = soup.find('table',{'class':'wikitable sortable'})
for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
    
   
    if (index == 0):
        columns = section
    else:
        data.append(section)

ca_df = pd.DataFrame(data = data,columns = columns)
ca_df.rename(columns={'Postcode': 'PostalCode'}, inplace=True)
ca_df = ca_df[ca_df['Borough'] != 'Not assigned'].reset_index(drop=True)
c_df = ca_df.groupby(['PostalCode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
c_df.columns = ['PostalCode', 'Borough', 'Neighbourhood']


In [65]:
geo_data=pd.read_csv("https://cocl.us/Geospatial_data")
geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [66]:
geo_data.rename(columns={'Postal Code':'PostalCode'})
geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [70]:
for g,c in zip(geo_data['Postal Code'], ca_df['PostalCode']) :
    if(g==c):
        c_df['Latitude']=geo_data['Latitude']
        c_df['Longitude']=geo_data['Longitude']
    

In [73]:
c_df.head(15)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [72]:
c_df.shape

(103, 5)